# Job Requirements

# Imports, Global Variables & Utility Functions

In [137]:
# built-in packages
import os
import pickle
import time
from pprint import pprint
from typing import List, Dict, Tuple, Union, Optional, Any 
import posixpath
import json
import regex as re

# note (source: phind.ai): we are not using urllib due its unpredictability. 
#       for example, if the base url doesn't have a trailing slash,
#       urllib will not join the base url with the rel\ative url correctly.
#       Moreover, if the second url has a leading slash, urllib will ignore the base url.
# from urllib.parse import urljoin, 

# pip-installed packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
from benedict import benedict

In [117]:
# global variables
COUNTRIES = ['india', 'uae']
BASE_OUTPUT_URL = 'outputs'
EXCEL_DIR = 'excel_files'
CSV_DIR = 'csv_files'

# global constants
BASE_URL = "https://www.zomato.com/"

In [118]:
def save_pkl(content_to_be_saved, path) -> None:
    """
    Save the given content to a file using pickle serialization.

    Args:
        content_to_be_saved: Any serializable object.
        path (str): The full path to the file where the data will be saved.

    Example:
        >>> save_pkl({'name': 'John', 'age': 30}, 'data.pkl')

    Returns:
        None

    Notes:
        "path" could either mean relative path or absolute path.
    """
    with open(path, 'wb') as f:
        pickle.dump(content_to_be_saved, f)

def load_pkl(path:str) -> Union[bool, any]:
    """
    Load data from a file using pickle deserialization.

    Args:
        path (str): The full path to the file to be loaded.

    Example:
        >>> loaded_data = load_pkl('data.pkl')

    Returns:
        The loaded content if the file exists, or False if the file does not exist.

    Notes:
        "path" could either mean relative path or absolute path.
    """
    if not os.path.exists(path):
        return False
    with open(path, 'rb') as f:
        content = pickle.load(f)
    return content

def force_load_pkl(path:str, dtype:str='dict') -> any:
    """
    Load data from a file using pickle deserialization. If the file does not exist,
    create an empty file and return an empty dictionary or list based on the specified data type.

    Args:
        path (str): The full path to the file to be loaded or created.
        dtype (str, optional): The data type to create if the file does not exist ('dict' or 'list').

    Example:
        >>> loaded_data = force_load_pkl('data.pkl', dtype='dict')

    Returns:
        The loaded content if the file exists, or an empty dictionary or list if the file does not exist.

    Notes:
        "path" could either mean relative path or absolute path.
    """
    try:
        content = load_pkl(path)
        return content
    except Exception as e:
        if dtype == 'list':
            save_pkl([], path)
            return []
        else:
            save_pkl({}, path)
            return {}

def join_paths(base_path:str, *relative_paths:str) -> str:
    """
    Join two paths and normalize the result.

    Args:
        base_directory (str): The base directory path.
        relative_path (str): The relative path to be joined.

    Example:
        >>> result = join_paths('/user/documents', 'files/myfile.txt')

    Returns:
        The normalized and joined path.

    Notes:
        More about naming standards for path components here: https://stackoverflow.com/a/2235762
    """
    return os.path.normpath(os.path.join(base_directory, relative_path))

def change_sep(path:str, new_sep:str, old_sep:str=os.path.sep) -> str:
    """
    Change the path separator in a given path and normalize the result.

    Args:
        path (str): The input path.
        new_sep (str): The new path separator to use.
        old_sep (str, optional): The old path separator to replace (default is os.path.sep).

    Example:
        >>> result = change_sep('path/to/change', '/', '\\')

    Returns:
        The path with the new separator, normalized.

    Notes:
        Source 1: https://stackoverflow.com/questions/18707338/print-raw-string-from-variable-not-getting-the-answers#:~:text=To%20turn%20a%20variable%20to%20raw%20str%2C%20just%20use
        Source 2: https://mail.python.org/pipermail/tutor/2011-July/084788.html
    """
    return os.path.normpath(rf"{path}").replace(old_sep, new_sep)


In [119]:
def save_df(df:pd.DataFrame, file_name_without_ext:str, to_excel:bool=True, to_csv:bool=True) -> None:
    """
    Save a dataframe to excel and csv files.
    """
    excel_path = join_paths(BASE_OUTPUT_URL, EXCEL_DIR, file_name_without_ext + '.xlsx')
    csv_path = join_paths(BASE_OUTPUT_URL, CSV_DIR, file_name_without_ext + '.csv')
    
    # check if the paths exist, and if not, create them
    if not os.path.exists(os.path.dirname(excel_path)):
        os.makedirs(os.path.dirname(excel_path))
    if not os.path.exists(os.path.dirname(csv_path)):
        os.makedirs(os.path.dirname(csv_path))

    if to_excel:
        df.to_excel(os.path.join(BASE_OUTPUT_URL, EXCEL_DIR, file_name_without_ext + '.xlsx'), index=False)
    if to_csv:
        df.to_csv(os.path.join(BASE_OUTPUT_URL, CSV_DIR, file_name_without_ext + '.csv'), index=False)

In [120]:
def recurse_convert_to_dict(json_dict:dict):
    
    if isinstance(json_dict, list):
        return [recurse_convert_to_dict(item) for item in json_dict]
    if not isinstance(json_dict, dict):
        return json_dict
    
    for k, v in json_dict.items():
        if isinstance(v, str):  # if the value is a string
            try:
                json_dict[k] = json.loads(v)  # try to convert it into a dict
                json_dict[k] = recurse_convert_to_dict(json_dict[k])  # and convert its strings as well
            except json.JSONDecodeError:  # if it's not a JSON string
                pass  # leave it as is
        elif isinstance(v, dict):  # if the value is a dict
            json_dict[k] = recurse_convert_to_dict(v)  # convert its strings
    return json_dict

def data_to_json(data:str):
    json_dict = json.loads(data)
    json_dict = recurse_convert_to_dict(json_dict)
    return json_dict

In [129]:
def dict_find_key(nested_dict, target_key):
    for key, value in nested_dict.items():
        if key == target_key:
            return value
        if isinstance(value, dict):
            result = dict_find_key(value, target_key)
            if result is not None:
                return result
    return None

def dict_find_and_replace(nested_dict, target_key, new_value):
    for key, value in nested_dict.items():
        if key == target_key:
            nested_dict[key] = new_value
        elif isinstance(value, dict):
            nested_dict[key] = dict_find_and_replace(value, target_key, new_value)
    return nested_dict


# Main Functions

# Main Code

In [121]:
for country in COUNTRIES:
    # create the country url
    country_url = posixpath.join(BASE_URL, country)
    print(f"Current Country URL: {country_url}")

    # use beautifulsoup to parse the html content
    country_html = requests.get(country_url, headers={'User-Agent': 'Mozilla/5.0'}).text
    country_soup = BeautifulSoup(country_html, 'html.parser')

    # get the list of cities
    cities = country_soup.select('div.states-container > div > div > div h5')
    cities = [city.text.strip().split(' Restaurants')[0] for city in cities]
    print(cities)
    
    # create the city url
    for i, city in enumerate(cities):
        city_url = posixpath.join(BASE_URL, city)
        print(f"Current City URL: {city_url}")
        break
    break


Current Country URL: https://www.zomato.com/india
['Agra', 'Ahmedabad', 'Ajmer', 'Alappuzha', 'Allahabad', 'Amravati', 'Amritsar', 'Aurangabad', 'Bengaluru', 'Bhopal', 'Bhubaneswar', 'Chandigarh', 'Chennai', 'Coimbatore', 'Cuttack', 'Darjeeling', 'Dehradun', 'Delhi NCR', 'Dharamshala', 'Gangtok', 'Goa', 'Gorakhpur', 'Guntur', 'Guwahati', 'Gwalior', 'Haridwar', 'Hyderabad', 'Indore', 'Jabalpur', 'Jaipur', 'Jalandhar', 'Jammu', 'Jamnagar', 'Jamshedpur', 'Jhansi', 'Jodhpur', 'Junagadh', 'Kanpur', 'Khajuraho', 'Khamgaon', 'Kharagpur', 'Kochi', 'Kolhapur', 'Kolkata', 'Kota', 'Lucknow', 'Ludhiana', 'Madurai', 'Manali', 'Mangalore', 'Manipal', 'Meerut', 'Mumbai', 'Mussoorie', 'Mysore', 'Nagpur', 'Nainital', 'Nashik', 'Neemrana', 'Ooty', 'Palakkad', 'Patiala', 'Patna', 'Puducherry', 'Pune', 'Pushkar', 'Raipur', 'Rajkot', 'Ranchi', 'Rishikesh', 'Salem', 'Shimla', 'Siliguri', 'Srinagar', 'Surat', 'Thrissur', 'Tirupati', 'Trichy', 'Trivandrum', 'Udaipur', 'Vadodara', 'Varanasi', 'Vellore', 'Vijay

In [122]:
import requests

cookies = {
    'PHPSESSID': '4b96bb7a198a94a216ff3c994d05a499',
    'fre': '0',
    'rd': '1380000',
    'zl': 'en',
    'fbtrack': '6afeb18328d78ee467b7f1fd3f8f952f',
    'uspl': 'true',
    'G_ENABLED_IDPS': 'google',
    'fbcity': '34',
    'ltv': '34',
    'lty': '34',
    'locus': '%7B%22addressId%22%3A0%2C%22lat%22%3A27.18%2C%22lng%22%3A78.02%2C%22cityId%22%3A34%2C%22ltv%22%3A34%2C%22lty%22%3A%22city%22%2C%22fetchFromGoogle%22%3Afalse%2C%22dszId%22%3A14225%2C%22fen%22%3A%22Agra%22%7D',
    'ak_bmsc': '3A3756FA2C3EAADCD1917A910E057D31~000000000000000000000000000000~YAAQlWp7XOyA0JWKAQAAtsWcrBW0d0YVMOzlKBqmlNubDQUUMFhLLHkAxymev9OAublvvSCEtLsGluFkcJl9dkTiHg2fhszMDLq0AwJ/VPa77AluDzD1yQGJVQIqA4ahXn7EHcKna+hq5pp4Jh3kKDMqNk7o4HXBIw9XcvQFd5V+8IOXAoVo0lPIsHN7GD2q3K/W2HlSPDI9cw/mdabPF3PrKm9luRyFLl6X9chmUW124STmwQ0x9tTBVbug0V5tVSzTnKpXlAJVhrPI0FRpimydPvVbE8vlXfbkw8eHITtnKApHgpwZ2uFfvMFNUQ+a347wSQKw742QtEyjLdA8y7AYjTzV4CgXptG//0wCmKFj9+r9rJwcwSRTAg==',
    'csrf': 'c7dc25cc63e3cac1235af2110cb2f597',
    'AWSALBTG': 'ffYYXrjQyzjhb8ij6ptwK4RPVbofNBnJTHAmvAB6XTE1zQrzwbKo6mRAUOo5G2J7GoqN/nlucvRIAQBTZQ+JnokQA/UicsrZPQGKCpGJMR/QmGEpLsIQ2Yq9nFn1c3FHRA9ziEeANUp2qDPVbJ4pvl/qOjsi0KKR2nmjsJzJ3J2a',
    'AWSALBTGCORS': 'ffYYXrjQyzjhb8ij6ptwK4RPVbofNBnJTHAmvAB6XTE1zQrzwbKo6mRAUOo5G2J7GoqN/nlucvRIAQBTZQ+JnokQA/UicsrZPQGKCpGJMR/QmGEpLsIQ2Yq9nFn1c3FHRA9ziEeANUp2qDPVbJ4pvl/qOjsi0KKR2nmjsJzJ3J2a',
}

headers = {
    'authority': 'www.zomato.com',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9,en-GB;q=0.8',
    'content-type': 'application/json',
    # 'cookie': 'PHPSESSID=4b96bb7a198a94a216ff3c994d05a499; fre=0; rd=1380000; zl=en; fbtrack=6afeb18328d78ee467b7f1fd3f8f952f; uspl=true; G_ENABLED_IDPS=google; fbcity=34; ltv=34; lty=34; locus=%7B%22addressId%22%3A0%2C%22lat%22%3A27.18%2C%22lng%22%3A78.02%2C%22cityId%22%3A34%2C%22ltv%22%3A34%2C%22lty%22%3A%22city%22%2C%22fetchFromGoogle%22%3Afalse%2C%22dszId%22%3A14225%2C%22fen%22%3A%22Agra%22%7D; ak_bmsc=3A3756FA2C3EAADCD1917A910E057D31~000000000000000000000000000000~YAAQlWp7XOyA0JWKAQAAtsWcrBW0d0YVMOzlKBqmlNubDQUUMFhLLHkAxymev9OAublvvSCEtLsGluFkcJl9dkTiHg2fhszMDLq0AwJ/VPa77AluDzD1yQGJVQIqA4ahXn7EHcKna+hq5pp4Jh3kKDMqNk7o4HXBIw9XcvQFd5V+8IOXAoVo0lPIsHN7GD2q3K/W2HlSPDI9cw/mdabPF3PrKm9luRyFLl6X9chmUW124STmwQ0x9tTBVbug0V5tVSzTnKpXlAJVhrPI0FRpimydPvVbE8vlXfbkw8eHITtnKApHgpwZ2uFfvMFNUQ+a347wSQKw742QtEyjLdA8y7AYjTzV4CgXptG//0wCmKFj9+r9rJwcwSRTAg==; csrf=c7dc25cc63e3cac1235af2110cb2f597; AWSALBTG=ffYYXrjQyzjhb8ij6ptwK4RPVbofNBnJTHAmvAB6XTE1zQrzwbKo6mRAUOo5G2J7GoqN/nlucvRIAQBTZQ+JnokQA/UicsrZPQGKCpGJMR/QmGEpLsIQ2Yq9nFn1c3FHRA9ziEeANUp2qDPVbJ4pvl/qOjsi0KKR2nmjsJzJ3J2a; AWSALBTGCORS=ffYYXrjQyzjhb8ij6ptwK4RPVbofNBnJTHAmvAB6XTE1zQrzwbKo6mRAUOo5G2J7GoqN/nlucvRIAQBTZQ+JnokQA/UicsrZPQGKCpGJMR/QmGEpLsIQ2Yq9nFn1c3FHRA9ziEeANUp2qDPVbJ4pvl/qOjsi0KKR2nmjsJzJ3J2a',
    'origin': 'https://www.zomato.com',
    'referer': 'https://www.zomato.com/agra',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Google Chrome";v="116"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
    'x-zomato-csrft': 'c7dc25cc63e3cac1235af2110cb2f597',
}

json_data = {
    'context': 'delivery',
    'filters': '{"searchMetadata":{"previousSearchParams":"{\\"PreviousSearchId\\":\\"3b79eef9-c768-4d84-b487-2a1fa60aae2f\\",\\"PreviousSearchFilter\\":[\\"{\\\\\\"category_context\\\\\\":\\\\\\"delivery_home\\\\\\"}\\",\\"\\"]}","postbackParams":"{\\"processed_chain_ids\\":[19748876,3400169,19468292,19183764,20470976,19461231,20129295,3400105,19382015],\\"shown_res_count\\":9,\\"search_id\\":\\"3b79eef9-c768-4d84-b487-2a1fa60aae2f\\"}","totalResults":1455,"hasMore":true,"getInactive":false},"dineoutAdsMetaData":{},"appliedFilter":[{"filterType":"category_sheet","filterValue":"delivery_home","isHidden":true,"isApplied":true,"postKey":"{\\"category_context\\":\\"delivery_home\\"}"}],"urlParamsForAds":{}}',
    'addressId': 0,
    'entityId': 34,
    'entityType': 'city',
    'locationType': '',
    'isOrderLocation': 1,
    'cityId': 34,
    'latitude': '27.1800000000000000',
    'longitude': '78.0200000000000000',
    'userDefinedLatitude': 27.18,
    'userDefinedLongitude': 78.02,
    'entityName': 'Agra',
    'orderLocationName': 'Agra',
    'cityName': 'Agra',
    'countryId': 1,
    'countryName': 'India',
    'displayTitle': 'Agra',
    'o2Serviceable': True,
    'placeId': '14225',
    'cellId': '4140058016516931584',
    'deliverySubzoneId': 14225,
    'placeType': 'DSZ',
    'placeName': 'Agra',
    'isO2City': True,
    'fetchFromGoogle': False,
    'fetchedFromCookie': True,
    'isO2OnlyCity': False,
    'address_template': [],
    'otherRestaurantsUrl': '',
}

response = requests.post('https://www.zomato.com/webroutes/search/home', cookies=cookies, headers=headers, json=json_data)

# Note: json_data will not be serialized by requests
# exactly as it was in the original request.
#data = '{"context":"delivery","filters":"{\\"searchMetadata\\":{\\"previousSearchParams\\":\\"{\\\\\\"PreviousSearchId\\\\\\":\\\\\\"3b79eef9-c768-4d84-b487-2a1fa60aae2f\\\\\\",\\\\\\"PreviousSearchFilter\\\\\\":[\\\\\\"{\\\\\\\\\\\\\\"category_context\\\\\\\\\\\\\\":\\\\\\\\\\\\\\"delivery_home\\\\\\\\\\\\\\"}\\\\\\",\\\\\\"\\\\\\"]}\\",\\"postbackParams\\":\\"{\\\\\\"processed_chain_ids\\\\\\":[19748876,3400169,19468292,19183764,20470976,19461231,20129295,3400105,19382015],\\\\\\"shown_res_count\\\\\\":9,\\\\\\"search_id\\\\\\":\\\\\\"3b79eef9-c768-4d84-b487-2a1fa60aae2f\\\\\\"}\\",\\"totalResults\\":1455,\\"hasMore\\":true,\\"getInactive\\":false},\\"dineoutAdsMetaData\\":{},\\"appliedFilter\\":[{\\"filterType\\":\\"category_sheet\\",\\"filterValue\\":\\"delivery_home\\",\\"isHidden\\":true,\\"isApplied\\":true,\\"postKey\\":\\"{\\\\\\"category_context\\\\\\":\\\\\\"delivery_home\\\\\\"}\\"}],\\"urlParamsForAds\\":{}}","addressId":0,"entityId":34,"entityType":"city","locationType":"","isOrderLocation":1,"cityId":34,"latitude":"27.1800000000000000","longitude":"78.0200000000000000","userDefinedLatitude":27.18,"userDefinedLongitude":78.02,"entityName":"Agra","orderLocationName":"Agra","cityName":"Agra","countryId":1,"countryName":"India","displayTitle":"Agra","o2Serviceable":true,"placeId":"14225","cellId":"4140058016516931584","deliverySubzoneId":14225,"placeType":"DSZ","placeName":"Agra","isO2City":true,"fetchFromGoogle":false,"fetchedFromCookie":true,"isO2OnlyCity":false,"address_template":[],"otherRestaurantsUrl":""}'
#response = requests.post('https://www.zomato.com/webroutes/search/home', cookies=cookies, headers=headers, data=data)

In [138]:
resp = requests.get('https://www.zomato.com/agra', headers={'User-Agent': 'Mozilla/5.0'})
resp.status_code

200

In [156]:
pattern = r'\\"resId\\":(\d+)'

# Use re.findall to extract text between JSON.parse(...)
matches = re.findall(pattern, resp.text)
matches

['18966194',
 '19857789',
 '20618516',
 '3400279',
 '19468292',
 '20137345',
 '20295169',
 '3400005',
 '20431659',
 '0']

In [131]:
new_json_data = dict_find_and_replace(json_data, 'shown_res_count', 100)

In [132]:
response = requests.post('https://www.zomato.com/webroutes/search/home', cookies=cookies, headers=headers, json=new_json_data)

In [136]:
len(response.json()['sections']['SECTION_SEARCH_RESULT'])

12

In [125]:
pprint(benedict(recurse_convert_to_dict(json_data))['filters'])

{'appliedFilter': [{'filterType': 'category_sheet',
                    'filterValue': 'delivery_home',
                    'isApplied': True,
                    'isHidden': True,
                    'postKey': '{"category_context":"delivery_home"}'}],
 'dineoutAdsMetaData': {},
 'searchMetadata': {'getInactive': False,
                    'hasMore': True,
                    'postbackParams': {'processed_chain_ids': [19748876,
                                                               3400169,
                                                               19468292,
                                                               19183764,
                                                               20470976,
                                                               19461231,
                                                               20129295,
                                                               3400105,
                                                          

In [58]:
for key, val in response.headers.items():
    print(key)
    print(f'\t{val}')

Content-Type
	application/json;charset=utf-8
Content-Length
	4477
Server
	envoy
x-xss-protection
	1; mode=block; report=https://www.zomato.com/cspreport.php
Content-Security-Policy
	frame-ancestors 'self' https://microapps.google.com/ https://pwa.zomato.com; default-src *; font-src * data:; child-src 'self' * blob:; img-src * 'self' data: blob: ; media-src * blob:; script-src 'self' 'unsafe-inline' 'unsafe-eval' browser.sentry-cdn.com *.jwpcdn.com *.licdn.com *.cloudflare.com *.twitter.com *.recruiterbox.com *.zdev.net *.zdev.net:8080 *.zomato.com *.tinymce.com *.gstatic.com *.googleapis.com *.google.com *.google.co.in *.facebook.com sdk.accountkit.com *.doubleclick.net *.googletagservices.com *.googlesyndication.com *.nr-data.net *.newrelic.com *.google-analytics.com *.zmtcdn.com *.googletagmanager.com *.facebook.net *.googleadservices.com *.cdninstagram.com *.googlesyndication.com *.spreedly.com *.instagram.com *.twimg.com d1m0gkspj3l6or.cloudfront.net d3mvnvhjmkxpjz.cloudfront.net d

In [51]:
benedict(json.loads(response.text))['sections.SECTION_SEARCH_RESULT[0].info']

{'resId': 3401744, 'name': 'The Salt Cafe Kitchen & Bar', 'image': {'url': 'https://b.zmtcdn.com/data/pictures/4/3401744/a5fe55d075d83d4c4d9915f2776d940e_featured_v2.jpg', 'urlWithParams': 'https://b.zmtcdn.com/data/pictures/4/3401744/a5fe55d075d83d4c4d9915f2776d940e_featured_v2.jpg?fit=around%7C108%3A108&crop=108%3A108%3B%2A%2C%2A'}, 'o2FeaturedImage': {'url': 'https://b.zmtcdn.com/data/pictures/4/3401744/3ddba5d2589a76c62fdcd9d8c8eaefc4_o2_featured_v2.jpg'}, 'rating': {'has_fake_reviews': 0, 'aggregate_rating': '4.6', 'rating_text': '4.6', 'rating_subtitle': 'Excellent', 'rating_color': '3F7E00', 'votes': '7,328', 'subtext': 'REVIEWS', 'is_new': False}, 'ratingNew': {'newlyOpenedObj': None, 'suspiciousReviewObj': None, 'ratings': {'DINING': {'rating_type': 'DINING', 'rating': '4.9', 'reviewCount': '5,296', 'reviewTextSmall': '5,296 Reviews', 'subtext': '5,296 Dining Reviews', 'color': '#1C1C1C', 'ratingV2': '4.9', 'subtitle': 'DINING', 'sideSubTitle': 'Dining Reviews', 'bgColorV2': {

In [28]:
city_html = requests.post('https://www.zomato.com/agra', ).text


'Visakhapatnam Restaurantschevron-right'

# Visualizing Functions' Return Values